In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/df_classified_with_topics.csv" )
dataset.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,post_id,post_text,category,time,timestamp,likes,comments,shares,username,is_live,reaction_count,Like,Love,Wow,Sad,Angry,Care,Weekday,Day,Month,Minute,Hour,Year
0,0,0,0,979693202944224,1200 كاش للربح ايقاف مقابلة ليون مارسيليا التف...,art-et-culture,1637593091000,2021-11-22 14:08:12,3.0,1,1,Radio Med Tunisie,False,NaN,0,0,0,0,0,0,Monday,22,11,58,14,2021
1,1,1,1,5243859242303198,اخماد حريق بنزل بالحمامات الشمالية,faits-divers,1637608280000,2021-11-22 18:11:20,12.0,0,0,Radio Med Tunisie,False,NaN,0,0,0,0,0,0,Monday,22,11,11,19,2021
2,2,2,2,5243473002341822,يتجدد موعدكم مروىالمعموري الاثنين للجمعة ال19 ...,art-et-culture,1637607301000,2021-11-22 17:55:01,2.0,0,0,Radio Med Tunisie,False,NaN,0,0,0,0,0,0,Monday,22,11,55,18,2021
3,3,3,3,5243801252308997,تونس تستضيف فعالية صنع ليبيا لدعم شراكة البلدين,politique,1637606782000,2021-11-22 17:46:22,5.0,0,1,Radio Med Tunisie,False,NaN,0,0,0,0,0,0,Monday,22,11,46,18,2021
4,4,4,4,396392812167201,نقترحو عليكم سلسلة تاريخية أنجح الأعمال الكوري...,art-et-culture,1637605403000,2021-11-22 17:30:16,6.0,0,2,Radio Med Tunisie,False,NaN,0,0,0,0,0,0,Monday,22,11,23,18,2021


In [ ]:
dataset.drop(columns=["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","post_id",'time','likes','comments','shares','username','is_live','reaction_count','Like','Love','Wow','Sad','Angry','Care','Year'],inplace=True)

In [ ]:
dataset.head()

,post_text,category,timestamp,Weekday,Day,Month,Minute,Hour
0,1200 كاش للربح ايقاف مقابلة ليون مارسيليا التف...,art-et-culture,2021-11-22 14:08:12,Monday,22,11,58,14
1,اخماد حريق بنزل بالحمامات الشمالية,faits-divers,2021-11-22 18:11:20,Monday,22,11,11,19
2,يتجدد موعدكم مروىالمعموري الاثنين للجمعة ال19 ...,art-et-culture,2021-11-22 17:55:01,Monday,22,11,55,18
3,تونس تستضيف فعالية صنع ليبيا لدعم شراكة البلدين,politique,2021-11-22 17:46:22,Monday,22,11,46,18
4,نقترحو عليكم سلسلة تاريخية أنجح الأعمال الكوري...,art-et-culture,2021-11-22 17:30:16,Monday,22,11,23,18


In [ ]:
pd.get_dummies(dataset)

,Day,Month,Minute,Hour,post_text_02 02 12,post_text_03 رقم شنوة,post_text_06 34 0,post_text_0أنيس خرباش بلاغات وزارة التجارة شعبوية ومغلوطةأنيس خرباش أسبوع ستتراجع أسعار الخضرأنيس خرباش 80 المنتفعين بالأراضي الدولية يستغلونها وتحصلوا عليها التهرب الضريبيالطبوبي يدعو رئيس الجمهورية ضرورة إصدار قانون يجر الاعتداء المرب والأطب اءالطبوبي صفاقس أصبحت جهة منكوبةالط الشرعي الأرجح تك ون وفاة الشاب بعقارب طبيعي والفقيد يعاني مرض بالقلبهذا جاء اعترافات التلميذ اعتدى أستاذه بسكين وساطور مصري قاضي زوجته خدعتني بالمكياج,post_text_1,post_text_1 11001300 15001700,post_text_1 2020,post_text_1 6,post_text_1 7,post_text_1 أخر أخبار الاولمبي الباجي المراسل علاء التوكابري1,post_text_1 الفلكي عمار الورتاني يكشف أسرار خفايا الأبراج,post_text_10,post_text_10 12,post_text_10 12 25 38,post_text_10 34 0,post_text_10 أشخاص فقط سهرة رأس العامالخرجة أمين,post_text_10 سنوات سجنا ضد بلحسن الطرابلسي و8 سنوات ضد سامي الفهري وخطية 40 مليون دينارتراجع حاد إنتاج الفسفاط التجاري بولاية قفصة بداية السنة,post_text_10 سنين لإعادة تهيئة مستوصف لتوة يخدموصلصوتك,post_text_10 طرق تحول طاقتك السلبية الى ايجابية,post_text_10 منتخبات متأهلة للدور الاخير تصفيات المونديال,post_text_100,post_text_100 1990 25 34 9900 13900 37 41 10900 14900,post_text_100 إطار طبي وشبه طبي القطاعين العام والخاص يخضعون الحجر الصحيمعبعضناغدوةخير 19,post_text_100 سنة الحركة الشيوعية بالبلاد التونسيةدحبيب كزدغلي مؤرخ العميد السابق للكلية الاداب بمنوبة,post_text_100 منقطع الدراسة سنويا تونس وفق تقرير جديد وزارة التجارة تغيير العلاقة التجارية تونس بريطانيا البريكسيت خولة الكعبي,post_text_1000 رحلة ملغاة مطار بكين مدارس تغلق ودعوة لعدم مغادرة البلاد,post_text_1087 اصابة 11 وفاة جديدة بتاريخ 24 لأول مرة وزارة الصحة تنبه الي خطورة الوضع 10 معتمدياتوزير الصحة يؤكد فرض الحجر الصحي الشامل المعتمديات الحالة,post_text_11 11 21 31,post_text_11 34 0,post_text_11 حالة وفاة غرقا الشواطئ التونسية 1 الجاري سيف الدين المهري,post_text_11 سنة ايقاف عماد الطرابلسي ابرام هيئة الحقيقة الكرامة عماد الطرابلسي اتفاق مبدئي بالصلح وين وصل الملف شنوة الجديد الاتفاقية التحكيمية,post_text_110 000,post_text_112 2016,post_text_114 إصابة مؤك دة بالكورونا تونس ثبوت 25 إصابة جديدة اليوم 19,post_text_12,post_text_12 2021 1400,...,timestamp_2021-11-22 13:20:55,timestamp_2021-11-22 13:55:01,timestamp_2021-11-22 14:08:12,timestamp_2021-11-22 14:15:05,timestamp_2021-11-22 14:19:04,timestamp_2021-11-22 14:23:02,timestamp_2021-11-22 14:41:06,timestamp_2021-11-22 15:11:14,timestamp_2021-11-22 15:14:27,timestamp_2021-11-22 15:32:58,timestamp_2021-11-22 15:37:31,timestamp_2021-11-22 15:52:12,timestamp_2021-11-22 15:54:11,timestamp_2021-11-22 16:00:04,timestamp_2021-11-22 16:12:40,timestamp_2021-11-22 16:14:20,timestamp_2021-11-22 16:31:30,timestamp_2021-11-22 16:35:32,timestamp_2021-11-22 16:51:35,timestamp_2021-11-22 16:51:39,timestamp_2021-11-22 17:14:30,timestamp_2021-11-22 17:21:03,timestamp_2021-11-22 17:30:16,timestamp_2021-11-22 17:46:22,timestamp_2021-11-22 17:55:01,timestamp_2021-11-22 18:11:20,timestamp_2021-11-22 18:31:06,timestamp_2021-11-22 18:43:26,timestamp_2021-11-22 19:00:04,timestamp_2021-11-22 19:00:10,timestamp_2021-11-22 19:02:41,timestamp_2021-11-22 19:15:38,timestamp_2021-11-22 19:16:40,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
0,22,11,58,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,22,11,11,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,22,11,55,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,22,11,46,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,22,11,23,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
x = dataset[['Weekday','Day','Month','Minute','Hour']]
y= dataset['category']

In [ ]:
x.head()

,Weekday,Day,Month,Minute,Hour
0,Monday,22,11,58,14
1,Monday,22,11,11,19
2,Monday,22,11,55,18
3,Monday,22,11,46,18
4,Monday,22,11,23,18


In [ ]:
pd.get_dummies(x)

,Day,Month,Minute,Hour,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
0,22,11,58,14,0,1,0,0,0,0,0
1,22,11,11,19,0,1,0,0,0,0,0
2,22,11,55,18,0,1,0,0,0,0,0
3,22,11,46,18,0,1,0,0,0,0,0
4,22,11,23,18,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
15361,27,11,15,3,0,0,1,0,0,0,0
15362,27,11,15,4,0,0,1,0,0,0,0
15363,27,11,15,6,0,0,1,0,0,0,0
15364,27,11,15,6,0,0,1,0,0,0,0


In [ ]:
y.head()

0    art-et-culture
1      faits-divers
2    art-et-culture
3         politique
4    art-et-culture
Name: category, dtype: object

In [ ]:
pd.get_dummies(y)

,art-et-culture,economie,faits-divers,medias,orbites,politique,regions,societe,sport
0,1,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
15361,0,0,0,1,0,0,0,0,0
15362,1,0,0,0,0,0,0,0,0
15363,1,0,0,0,0,0,0,0,0
15364,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
x_encoded =OneHotEncoder().fit_transform(x)

In [ ]:
x_encoded.data

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
y.head()

0    art-et-culture
1      faits-divers
2    art-et-culture
3         politique
4    art-et-culture
Name: category, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(x_encoded,y,test_size=0.2,stratify=y,random_state=20)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
le = preprocessing.LabelEncoder()

In [ ]:
scaler = StandardScaler(with_mean=False)
scaler.fit_transform(X_train)
scaler.transform(X_val)

<3074x131 sparse matrix of type '<class 'numpy.float64'>'
	with 15370 stored elements in Compressed Sparse Row format>

In [ ]:
X_val.data

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
from sklearn import neighbors
KNN_model=neighbors.KNeighborsClassifier(n_neighbors=1,n_jobs=-1)
KNN_model.fit(X_train,y_train)

KNeighborsClassifier(n_jobs=-1, n_neighbors=1)

In [ ]:
pred=KNN_model.predict(X_val)
print("Accuracy={}%".format((sum(y_val==pred)/y_val.shape[0])*100))

Accuracy=40.27325959661678%


In [ ]:
from sklearn import neighbors 
from sklearn.metrics import f1_score,confusion_matrix,roc_auc_score
f1_list=[]
k_list=[]
for k in range(1,10):
    clf=neighbors.KNeighborsClassifier(n_neighbors=k,n_jobs=-1)
    clf.fit(X_train,y_train)
    pred=clf.predict(X_val)
    f=f1_score(y_val,pred,average='macro')
    f1_list.append(f)
    k_list.append(k)

In [ ]:
best_f1_score=max(f1_list)
best_k=k_list[f1_list.index(best_f1_score)]        
print("Optimum K value=",best_k," with F1-Score=",best_f1_score)

Optimum K value= 1  with F1-Score= 0.2219121791889752


# Logistic Reg

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import classification_report

##testModel function will take as input the model name, X: feature and y: label
#We automated the testing model part because it will save us time when testing multiple models such as
#LogisticRegression + SGDClassifier + MultinomialNB + RandomForestClassifier

def testModel(model,X_train,X_test,y_train, y_test):    
    model = model()
    model.fit(X_train,y_train)
    modelName = type(model).__name__
    pred=model.predict(X_test)
    print(modelName)
    print(classification_report(y_test,model.predict(X_test)))
    score=np.mean(cross_val_score(model, X, y, cv=5))

    return model,{"model":modelName,"score":score}

In [ ]:
from sklearn.linear_model import LogisticRegression
testModel(LogisticRegression,X_train,X_val,y_train,y_val)

LogisticRegression
                precision    recall  f1-score   support

art-et-culture       0.54      0.99      0.69      1640
      economie       0.00      0.00      0.00       180
  faits-divers       0.00      0.00      0.00        68
        medias       0.27      0.05      0.08        88
       orbites       0.00      0.00      0.00       234
     politique       0.00      0.00      0.00       171
       regions       0.00      0.00      0.00       190
       societe       0.00      0.00      0.00       215
         sport       0.35      0.05      0.09       288

      accuracy                           0.53      3074
     macro avg       0.13      0.12      0.10      3074
  weighted avg       0.33      0.53      0.38      3074



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

(LogisticRegression(), {'model': 'LogisticRegression', 'score': nan})

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(penalty='l2',C=10,max_iter=400,solver='lbfgs',verbose=3)
model.fit(X_train,y_train)
pred=model.predict(X_val)
print(classification_report(y_val,model.predict(X_val)))
score=np.mean(cross_val_score(model, x, y, cv=5))
print(score)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


                precision    recall  f1-score   support

art-et-culture       0.54      0.98      0.70      1640
      economie       0.00      0.00      0.00       180
  faits-divers       0.00      0.00      0.00        68
        medias       0.30      0.08      0.13        88
       orbites       0.00      0.00      0.00       234
     politique       0.00      0.00      0.00       171
       regions       0.00      0.00      0.00       190
       societe       0.00      0.00      0.00       215
         sport       0.37      0.06      0.10       288

      accuracy                           0.53      3074
     macro avg       0.13      0.12      0.10      3074
  weighted avg       0.33      0.53      0.38      3074

nan


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s finished
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete